### Aggregate Data

#### Setup

In [1]:
import pandas as pd
import numpy as np
path = 'individual-datasets/temp-precip/'

In [3]:
counties = ['accomack', 'fauquier', 'greensville', 'hanover', 'rockingham', 'wise']
temp_dfs = {}
precip_dfs = {}
for county in counties:
    temp = pd.read_csv(path+county+'_t'+'.csv', skiprows=4)
    precip = pd.read_csv(path+county+'_p'+'.csv', skiprows=4)
    temp_dfs[county] = temp
    precip_dfs[county] = precip

#### Goals:
- ~change months from observations to columns~
- ~add column for county and combine on year and county~
- ~combine dfs~
- ~make income-dsci table~
- ~combine with income-dsci~
- ~combine sectors use total as value for VACO2~
- ~remap crop to categories and make categories columns~
- ~merge everything together~

#### Change months from observations to columns

In [4]:
for county in counties:
    for dfs in [temp_dfs, precip_dfs]:
        dfs[county]['Month'] = [int(str(s)[-2:]) for s in dfs[county].Date]
        dfs[county]['Year'] = [str(s)[:4] for s in dfs[county].Date]
        dfs[county] = dfs[county].drop(['Anomaly', 'Date'], axis=1)
        dfs[county] = dfs[county].pivot_table(index=['Year'], 
                                      columns='Month', 
                                      values='Value')
        dfs[county] = dfs[county].reset_index()
        dfs[county]['County'] = county.title()

#### Merge tables

In [6]:
temps = pd.concat(temp_dfs).reset_index().drop(['level_0', 'level_1'], axis=1)
temps = temps.rename({i:'T'+str(i) for i in range(1, 13)}, axis=1)
precips = pd.concat(precip_dfs).reset_index().drop(['level_0', 'level_1'], axis=1)
precips = precips.rename({i:'P'+str(i) for i in range(1, 13)}, axis=1)

In [7]:
merged = pd.merge(precips, temps, on=['County', 'Year'])\
            .set_index(['County', 'Year']).reset_index()

In [8]:
inc_dsci = pd.read_csv('income-dsci-temp-data.csv')
inc_dsci = inc_dsci.drop(['Precip', 'Temp'], axis=1)
inc_dsci['Year'] = inc_dsci['Year'].astype(str)

In [9]:
df = pd.merge(inc_dsci, merged, on=['County', 'Year'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90 entries, 0 to 89
Data columns (total 28 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   County  90 non-null     object 
 1   Year    90 non-null     object 
 2   Income  90 non-null     int64  
 3   DSCI    90 non-null     float64
 4   P1      90 non-null     float64
 5   P2      90 non-null     float64
 6   P3      90 non-null     float64
 7   P4      90 non-null     float64
 8   P5      90 non-null     float64
 9   P6      90 non-null     float64
 10  P7      90 non-null     float64
 11  P8      90 non-null     float64
 12  P9      90 non-null     float64
 13  P10     90 non-null     float64
 14  P11     90 non-null     float64
 15  P12     90 non-null     float64
 16  T1      90 non-null     float64
 17  T2      90 non-null     float64
 18  T3      90 non-null     float64
 19  T4      90 non-null     float64
 20  T5      90 non-null     float64
 21  T6      90 non-null     float64
 22  T7  

In [11]:
co2 = pd.read_csv('individual-datasets/VA_CO2Emissions.csv')
co2 = co2[['Year', 'Total', 'Sector']]
co2

,Year,Total,Sector
0,1970,9.664151,Residential
1,1971,9.443978,Residential
2,1972,9.583187,Residential
3,1973,9.118182,Residential
4,1974,8.245722,Residential
...,...,...,...
307,2017,105.584974,Total
308,2018,110.056786,Total
309,2019,106.794626,Total
310,2020,98.175849,Total


In [12]:
co2 = co2.pivot_table(index=['Year'],columns='Sector', values='Total').reset_index()
co2.Year = co2.Year.astype(str)
co2 = co2.rename({'Total': 'TotalCO2'})
co2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 52 entries, 0 to 51
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Year            52 non-null     object 
 1   Commercial      52 non-null     float64
 2   Electic         52 non-null     float64
 3   Industrial      52 non-null     float64
 4   Residential     52 non-null     float64
 5   Total           52 non-null     float64
 6   Transportation  52 non-null     float64
dtypes: float64(6), object(1)
memory usage: 3.2+ KB


In [13]:
df = pd.merge(df, co2, on=['Year'], how='left')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90 entries, 0 to 89
Data columns (total 34 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   County          90 non-null     object 
 1   Year            90 non-null     object 
 2   Income          90 non-null     int64  
 3   DSCI            90 non-null     float64
 4   P1              90 non-null     float64
 5   P2              90 non-null     float64
 6   P3              90 non-null     float64
 7   P4              90 non-null     float64
 8   P5              90 non-null     float64
 9   P6              90 non-null     float64
 10  P7              90 non-null     float64
 11  P8              90 non-null     float64
 12  P9              90 non-null     float64
 13  P10             90 non-null     float64
 14  P11             90 non-null     float64
 15  P12             90 non-null     float64
 16  T1              90 non-null     float64
 17  T2              90 non-null     float

In [14]:
crop = pd.read_csv('individual-datasets/crop_data.csv')
crop_replace_map = {
    'Corn': 'Agriculture',
    'Other_Hay/Non_Alfalfa': 'Agriculture',
    'Soybeans': 'Agriculture',
    'Alfalfa':'Agriculture',
    'Sorghum':'Agriculture',
    'Dbl_Crop_WinWht/Soybeans':'Agriculture',
    'Winter_Wheat':'Agriculture',
    'Dbl_Crop_WinWht/Corn':'Agriculture',
    'Millet':'Agriculture',
    'Dbl_Crop_Barley/Soybeans':'Agriculture',
    'Rye':'Agriculture',
    'Barley':'Agriculture',
    'Dbl_Crop_Soybeans/Oats ':'Agriculture',
    'Dbl_Crop_Barley/Corn':'Agriculture',
    'Other_Crops':'Agriculture',
    'Dbl_Crop_WinWht/Sorghum':'Agriculture',
    'Oats':'Agriculture',
    'Cotton':'Agriculture',
    'Apples':'Agriculture',
    'Pumpkins':'Agriculture',
    'Tobacco':'Agriculture',
    'Sunflower':'Agriculture',
    'Christmas_Trees':'Agriculture',
    'Triticale':'Agriculture',
    'Grapes':'Agriculture',
    'Sweet_Corn':'Agriculture',
    'Peanuts':'Agriculture',
    'Dry_Beans':'Agriculture',
    'Dbl_Crop_Oats/Corn':'Agriculture',
    'Potatoes':'Agriculture',
    'Peaches':'Agriculture',
    'Misc_Vegs_&_Fruits':'Agriculture',
    'Dbl_Crop_Barley/Sorghum':'Agriculture',
    'Sweet_Potatoes':'Agriculture',
    'Squash':'Agriculture',
    'Tomatoes':'Agriculture',
    'Strawberries':'Agriculture',
    'Broccoli':'Agriculture',
    'Watermelons':'Agriculture',
    'Other_Tree_Crops':'Agriculture',
    'Canola':'Agriculture',
    'Cantaloupes':'Agriculture',
    'Cabbage':'Agriculture',
    'Vetch':'Agriculture',
    'Walnuts':'Agriculture',
    'Rape_Seed':'Agriculture',
    'Dbl_Crop_WinWht/Cotton':'Agriculture',
    'Dbl_Crop_Corn/Soybeans':'Agriculture',
    'Cherries':'Agriculture',
    'Buckwheat':'Agriculture',
    'Dbl_Crop_Soybeans/Cotton':'Agriculture',
    'Switchgrass':'Agriculture',
    'Cucumbers':'Agriculture',
    'Cranberries':'Agriculture',
    'Pop_or_Orn_Corn':'Agriculture',
    'Radishes':'Agriculture',
    'Greens':'Agriculture',
    'Pears':'Agriculture',
    'Peppers':'Agriculture',
    'Pecans':'Agriculture',
    'Peas':'Agriculture',
    'Durum_Wheat':'Agriculture',
    'Gourds':'Agriculture',
    'Spring_Wheat':'Agriculture',
    'Speltz':'Agriculture',
    'Turnips':'Agriculture',
    'Dbl_Crop_Soybeans/Oats':'Agriculture',

    'Grassland/Pasture': 'Grassland',
    'Sod/Grass_Seed': 'Grassland',
    'Switchgrass':'Grassland',
    'Clover/Wildflowers':'Grassland',

    'Mixed_Forest': 'Forest',
    'Evergreen_Forest': 'Forest',
    'Deciduous_Forest': 'Forest',

    'Barren': 'Barren',
    'Fallow/Idle_Cropland': 'Barren',

    'Developed/High_Intensity': 'Developed',
    'Developed/Med_Intensity': 'Developed',
    'Developed/Low_Intensity': 'Developed',
    'Developed/Open_Space': 'Developed',

    'Open_Water': 'Open_Water',
    'Shrubland': 'Shrubland',

    'Woody_Wetlands':'Wetlands',
    'Herbaceous_Wetlands':'Wetlands',

    'Aquaculture':'Aquaculture'

}

crop['Landuse'] = crop['Crop'].map(crop_replace_map)
crop = crop.drop(['Crop', 'Percentage'], axis=1)
crop

,Count,Year,County,Landuse
0,106,2008,Wise,Agriculture
1,156,2008,Wise,Agriculture
2,7,2008,Wise,Agriculture
3,16,2008,Wise,Agriculture
4,8399,2008,Wise,Agriculture
...,...,...,...,...
3281,18,2022,Rockingham,Agriculture
3282,227,2022,Rockingham,Agriculture
3283,6760,2022,Rockingham,Agriculture
3284,16,2022,Rockingham,Agriculture


In [15]:
crop_counts = pd.DataFrame(crop.groupby(['Year', 'County', 'Landuse']).agg({'Count':'sum'})).reset_index()
crop_counts = crop_counts.pivot(index=['Year','County'], columns='Landuse', values='Count')
crop_counts = crop_counts.fillna(0)
crop_counts.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 90 entries, (2008, 'Accomack') to (2022, 'Wise')
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Agriculture  90 non-null     float64
 1   Aquaculture  90 non-null     float64
 2   Barren       90 non-null     float64
 3   Developed    90 non-null     float64
 4   Forest       90 non-null     float64
 5   Grassland    90 non-null     float64
 6   Open_Water   90 non-null     float64
 7   Shrubland    90 non-null     float64
 8   Wetlands     90 non-null     float64
dtypes: float64(9)
memory usage: 7.5+ KB


#### change counts to percentages:
- find sum of counts for each county for each year
- replace count for each value with their percentage for that year

In [16]:
crop_counts['Total'] = crop_counts.sum(axis=1, numeric_only=True)

In [17]:
cols = [col for col in crop_counts.columns if col != 'Total']
cols

['Agriculture',
 'Aquaculture',
 'Barren',
 'Developed',
 'Forest',
 'Grassland',
 'Open_Water',
 'Shrubland',
 'Wetlands']

In [18]:
for col in cols:
    crop_counts[col] = crop_counts[col]/crop_counts['Total']
crop_counts

Landuse           Agriculture  Aquaculture    Barren  Developed    Forest  \
Year County                                                                 
2008 Accomack        0.328149     0.000000  0.012280   0.062278  0.032290   
     Fauquier        0.194887     0.000000  0.001739   0.082141  0.476392   
     Greensville     0.150240     0.000000  0.003702   0.066943  0.526131   
     Hanover         0.183223     0.000000  0.002972   0.109552  0.526886   
     Rockingham      0.176976     0.000000  0.000488   0.064113  0.590913   
...                       ...          ...       ...        ...       ...   
2022 Fauquier        0.161772     0.000000  0.003316   0.084908  0.475702   
     Greensville     0.167318     0.000001  0.001246   0.043400  0.503351   
     Hanover         0.179153     0.000000  0.002094   0.134504  0.525073   
     Rockingham      0.208160     0.000000  0.000551   0.083102  0.575540   
     Wise            0.015569     0.000000  0.029699   0.091445  0.790107   

Landuse           Grassland  Open_Water  Shrubland  Wetlands      Total  
Year County                                                              
2008 Accomack      0.005097    0.048362   0.009128  0.502416  1213063.0  
     Fauquier      0.233110    0.003266   0.003483  0.004981  1874001.0  
     Greensville   0.040477    0.004115   0.054518  0.153873   854114.0  
     Hanover       0.058362    0.004743   0.047442  0.066818  1364221.0  
     Rockingham    0.165885    0.001608   0.000003  0.000015  2455045.0  
...                     ...         ...        ...       ...        ...  
2022 Fauquier      0.250472    0.003982   0.005012  0.014836  1873531.0  
     Greensville   0.048971    0.005482   0.059341  0.170890   854110.0  
     Hanover       0.047976    0.005918   0.023318  0.081964  1364207.0  
     Rockingham    0.128402    0.002607   0.001482  0.000156  2442732.0  
     Wise          0.043779    0.002002   0.027274  0.000125  1165843.0  

[90 rows x 10 columns]

In [19]:
crop_counts = crop_counts.drop(['Total'], axis=1)
crop_counts

Landuse           Agriculture  Aquaculture    Barren  Developed    Forest  \
Year County                                                                 
2008 Accomack        0.328149     0.000000  0.012280   0.062278  0.032290   
     Fauquier        0.194887     0.000000  0.001739   0.082141  0.476392   
     Greensville     0.150240     0.000000  0.003702   0.066943  0.526131   
     Hanover         0.183223     0.000000  0.002972   0.109552  0.526886   
     Rockingham      0.176976     0.000000  0.000488   0.064113  0.590913   
...                       ...          ...       ...        ...       ...   
2022 Fauquier        0.161772     0.000000  0.003316   0.084908  0.475702   
     Greensville     0.167318     0.000001  0.001246   0.043400  0.503351   
     Hanover         0.179153     0.000000  0.002094   0.134504  0.525073   
     Rockingham      0.208160     0.000000  0.000551   0.083102  0.575540   
     Wise            0.015569     0.000000  0.029699   0.091445  0.790107   

Landuse           Grassland  Open_Water  Shrubland  Wetlands  
Year County                                                   
2008 Accomack      0.005097    0.048362   0.009128  0.502416  
     Fauquier      0.233110    0.003266   0.003483  0.004981  
     Greensville   0.040477    0.004115   0.054518  0.153873  
     Hanover       0.058362    0.004743   0.047442  0.066818  
     Rockingham    0.165885    0.001608   0.000003  0.000015  
...                     ...         ...        ...       ...  
2022 Fauquier      0.250472    0.003982   0.005012  0.014836  
     Greensville   0.048971    0.005482   0.059341  0.170890  
     Hanover       0.047976    0.005918   0.023318  0.081964  
     Rockingham    0.128402    0.002607   0.001482  0.000156  
     Wise          0.043779    0.002002   0.027274  0.000125  

[90 rows x 9 columns]

In [20]:
crop_counts = crop_counts.reset_index()

In [21]:
crop_counts.Year = crop_counts.Year.astype(str)

In [22]:
df = pd.merge(df, crop_counts, on=['Year', 'County'], how='left')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90 entries, 0 to 89
Data columns (total 43 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   County          90 non-null     object 
 1   Year            90 non-null     object 
 2   Income          90 non-null     int64  
 3   DSCI            90 non-null     float64
 4   P1              90 non-null     float64
 5   P2              90 non-null     float64
 6   P3              90 non-null     float64
 7   P4              90 non-null     float64
 8   P5              90 non-null     float64
 9   P6              90 non-null     float64
 10  P7              90 non-null     float64
 11  P8              90 non-null     float64
 12  P9              90 non-null     float64
 13  P10             90 non-null     float64
 14  P11             90 non-null     float64
 15  P12             90 non-null     float64
 16  T1              90 non-null     float64
 17  T2              90 non-null     float

In [24]:
df = df.drop(['Commercial', 'Electic', 'Industrial', 'Residential', 'Transportation'], axis=1)
df = df.rename({'Total' : 'VACO2'}, axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90 entries, 0 to 89
Data columns (total 38 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   County       90 non-null     object 
 1   Year         90 non-null     object 
 2   Income       90 non-null     int64  
 3   DSCI         90 non-null     float64
 4   P1           90 non-null     float64
 5   P2           90 non-null     float64
 6   P3           90 non-null     float64
 7   P4           90 non-null     float64
 8   P5           90 non-null     float64
 9   P6           90 non-null     float64
 10  P7           90 non-null     float64
 11  P8           90 non-null     float64
 12  P9           90 non-null     float64
 13  P10          90 non-null     float64
 14  P11          90 non-null     float64
 15  P12          90 non-null     float64
 16  T1           90 non-null     float64
 17  T2           90 non-null     float64
 18  T3           90 non-null     float64
 19  T4        

In [27]:
df.to_csv('county-year-agg.csv', index=False)